In [118]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from visualization import SvgGoBoard
from train_utils import LabelToCoord, ParseForPolicy, DrawGoBoard
from tensorflow.keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, Input, MaxPooling2D

tf.enable_eager_execution()

In [119]:
def ReadExamplesFromTFRecords(filename, max_num_examples=1000):
    """ Reads the first N examples from a TFRecord file.
    """
    # The file is compressed, by ../model/gen_dataset
    dataset = tf.data.TFRecordDataset([filename], 'ZLIB')
    examples = []
    examples_in_proto = []
    
    for ex in dataset:
        examples.append(ex)
        examples_in_proto.append(tf.train.Example.FromString(ex.numpy()))
        
    return examples, examples_in_proto

In [120]:
shusai_game, _ = ReadExamplesFromTFRecords('shusai_19380626.rio')
len(shusai_game)

235


## Basic Rules of Go

* Board, Players and Stones

* Liberties and Capturing

* End of Game

## How to Play Go

* Exhaustive Search

* Reducing Search Width

* Reducing Search Depth

## CNN and Go

* CNN

* Policy Network and Value Network

* Feature Set

* Search with Policy Network

* Monte-Carlo Tree Search with Value Network

### Board, Players and Stones

* $N\times N$ board (standard size: $19\times 19$)



* Two players (black and white) place stones alternatively on crosspoints



* Stones are not movable (but can be taken away if they are captured).

In [121]:
def RenderBasicBoard():
    bb = SvgGoBoard(9, 9, coord_style='A1')
    bb.AddStone(3, 3, 1)
    bb.AddStone(2, 2, -1)
    bb.AddStone(3, 2, 1)
    bb.AddStone(2, 3, -1)
    bb.AddStone(2, 4, 1)
    return bb

In [122]:
bb = RenderBasicBoard()
bb

### Liberties and Capturing

* Adjacent stones of the same color form a chain


* An unoccupied adjacent point of a chain is its __liberty__.


* A chain having no liberties is dead, and will be removed from the board immediately.

In [123]:
def RenderLiberty():
    bb = SvgGoBoard(9, 9, coord_style='A1')
    bb.AddStone(3, 3, 1)
    bb.AddStone(2, 2, -1)
    bb.AddStone(3, 2, 1)
    bb.AddStone(2, 3, -1)
    bb.AddStone(2, 4, 1)
    bb.AddSquare(2, 1, 0)
    bb.AddSquare(1, 2, 0)
    bb.AddSquare(1, 3, 0)
    return bb

In [124]:
bb = RenderLiberty()
bb

In [125]:
def RenderCapture():
    bb = SvgGoBoard(9, 9, coord_style='A1')
    bb.AddStone(3, 3, 1)
    bb.AddStone(2, 2, -1)
    bb.AddStone(3, 2, 1)
    bb.AddStone(2, 3, -1)
    bb.AddStone(2, 4, 1)
    bb.AddSquare(2, 1, 0)
    bb.AddStone(5, 5, -1)
    bb.AddStone(6, 6, -1)
    bb.AddStone(1, 2, 1)
    bb.AddStone(1, 3, 1)
    return bb

In [126]:
bb = RenderCapture()
bb

In [127]:
def RenderCaptured():
    bb = SvgGoBoard(9, 9, coord_style='A1')
    bb.AddStone(3, 3, 1)
    bb.AddStone(2, 1, 1)
    bb.AddStone(2, 4, 1)
    bb.AddStone(3, 2, 1)    
    bb.AddStone(5, 5, -1)
    bb.AddStone(6, 6, -1)
    bb.AddStone(1, 2, 1)
    bb.AddStone(1, 3, 1)
    return bb

In [128]:
bb = RenderCaptured()
bb

### Win or Loss

* A game ends when both players pass;

* The player who __controls__ more points wins the game.



In [129]:
def RenderGameEnding():
    bb = SvgGoBoard(9, 9, coord_style='A1')
    for k in range(9):
        bb.AddStone(k, 4, 1)
        bb.AddStone(k, 5, -1)
    bb.AddStone(2, 2, 1)
    bb.AddStone(5, 2, 1)
    bb.AddStone(7, 2, 1)
    bb.AddStone(1, 7, -1)
    bb.AddStone(3, 7, -1)
    bb.AddStone(6, 7, -1)
    return bb

In [130]:
bb = RenderGameEnding()
bb

In [131]:
def RenderRealExample():
    planes, label = ParseForPolicy(shusai_game[234])
    orig = planes.numpy()[:,:,0]
    bb = DrawGoBoard(orig)
    return bb

In [132]:
bb = RenderRealExample()
bb

### Exhaustive Search

* A natual tree structure

* Width: the number of possible moves

* Depth: the number of moves of a complete game

* $O(2^{170})$ nodes for a Go game on the standard board

### Reducing Search Width

* Normally 200~300 legal candidate moves at each step;

* Human players consider only 5-10 candidates for the first step;

* Trained human players have good intuition:

    * senstitive on shapes
    
    * think locally
    
    * remember hundreds of Josekis(定式)

In [133]:
def RenderEfficiencyExample():
    bb = SvgGoBoard(19, 19, coord_style='A1')
    bb.AddStone(3, 16, -1)
    bb.AddStone(2, 14, -1)
    bb.AddStone(9, 16, -1)
    bb.AddStone(9, 14, -1)
    bb.AddStone(10, 3, 1)
    bb.AddStone(10, 4, 1)
    bb.AddStone(11, 3, 1)
    bb.AddStone(11, 4, 1)
    return bb

In [134]:
bb = RenderEfficiencyExample()
bb

In [135]:
def RenderRelevanceExample():
    planes, label = ParseForPolicy(shusai_game[33])
    orig = planes.numpy()[:,:,0]
    bb = DrawGoBoard(orig)
    bb.AddSquare(2, 6, 0)
    bb.AddSquare(2, 7, 0)
    bb.AddSquare(3, 5, 0)
    bb.AddSquare(16, 9, -1)
    bb.AddSquare(15, 9, -1)    
    bb.AddSquare(9, 15, -1)
    return bb


In [136]:
bb = RenderRelevanceExample()
bb

In [137]:
def RenderJosekiExample():
    bb = SvgGoBoard(19, 19, coord_style='A1')
    bb.AddStone(3, 15, -1)
    bb.AddSquare(3, 15, 1)
    bb.AddStone(2, 13, 1)
    bb.AddSquare(2, 13, 2)
    bb.AddStone(5, 16, -1)
    bb.AddSquare(5, 16, 3)
    bb.AddStone(1, 15, 1)
    bb.AddSquare(1, 15, 4)
    bb.AddStone(2, 16, -1)
    bb.AddSquare(2, 16, 5)
    bb.AddStone(2, 10, 1)
    bb.AddSquare(2, 10, 6)
    return bb

In [138]:
bb = RenderJosekiExample()
bb

### Reducing Search Depth

* Humans can only search 20~40 steps;

* A heuristic valuation function for leaf nodes (that are far from the end of a game);

* Min-max rule: 
  * $score\_of\_node = 1 - max\{score\_of\_children\_nodes\}$


### Convolutional Neural Network

* Proven to be successful on image recognition, classification etc.

* A Go game state is a B/W image.

![](Typical_cnn.png "Typical CNN")

### Policy Network and Value Network

* Supervised training.

* Features:
  * $19\times 19\times 7$: image with 7 channels
  * Channel 1: the positions of black and white stones.
  * Channel 2-4: the positions of black chains with 1,2,3 liberties.
  * Channel 5-7: the positions of white chains with 1,2,3 liberties.
  
* Target:
  * Policy network: human player's next move.
  * Value network: final result of current player

In [139]:
def RenderFeatureChannel(k):
    planes, label = ParseForPolicy(shusai_game[123])
    bb = DrawGoBoard(planes.numpy()[:,:,k])
    return bb

In [140]:
bb = RenderFeatureChannel(0)
bb

In [141]:
bb = RenderFeatureChannel(1)
bb

In [142]:
bb = RenderFeatureChannel(2)
bb

In [143]:
bb = RenderFeatureChannel(3)
bb

### Model Structure


In [144]:
def BuildCommonLayers(input):
    x = Conv2D(filters=64, kernel_size=(7,7), activation="relu", padding="same",
               data_format='channels_last')(input)
    x = Conv2D(filters=64, kernel_size=(7,7), activation="relu", padding="same",
               data_format='channels_last')(x)
    x = Conv2D(filters=32, kernel_size=(5,5), activation="relu", padding="same",
               data_format='channels_last')(x)
    x = Conv2D(filters=32, kernel_size=(5,5), activation="relu", padding="same",
               data_format='channels_last')(x)
    x = Dropout(0.5)(x)
    x = Conv2D(filters=32, kernel_size=(5,5), activation="relu", padding="same",
               data_format='channels_last')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(512)(x)
    return x


def CompileModel(model):
    model.compile(optimizer="sgd", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    print(model.summary())
    return model


input = Input(shape=(19, 19, 7), name="go_input")
x = BuildCommonLayers(input)
x = Dense(19*19)(x)
policy_output = Activation("softmax", name="policy_output")(x)
model = keras.models.Model(inputs=input, outputs=[policy_output])
model = CompileModel(model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
go_input (InputLayer)        (None, 19, 19, 7)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 19, 19, 64)        22016     
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 19, 19, 64)        200768    
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 19, 19, 32)        51232     
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 19, 19, 32)        25632     
_________________________________________________________________
dropout_8 (Dropout)          (None, 19, 19, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 19, 19, 32)        25632     
__________

In [145]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
go_input (InputLayer)        (None, 19, 19, 7)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 19, 19, 64)        22016     
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 19, 19, 64)        200768    
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 19, 19, 32)        51232     
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 19, 19, 32)        25632     
_________________________________________________________________
dropout_8 (Dropout)          (None, 19, 19, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 19, 19, 32)        25632     
__________

### Example of Policy Network's output

* Trained with 20M training examples extracted from 90k pro games

* Accuracy (of top 10 candidates): 85%

* Observation:
  * Good intuition
  * Remember many Josekis

<img src="policy_output.png" width="600">

### Search With Policy Network

* Reducing search width by focusing on top 20 candidates.

* Search only 2 steps deeper.


### Evaluation of Leafs: MCTS with CNN

* Monte Carlo Tree Search: start from a leaf node, randomly play $N$ games (rollouts)

* Score of a leaf node: winning probability of current player.

* Value network: help terminate a rollout early